In [ ]:
import pandas as pd 
import numpy as np
import os
import json 
import random
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

In [2]:
train_data= pd.read_csv('data/train.csv')
test_data= pd.read_csv('data/test.csv')



In [4]:
train_data.head()

,valid_time,pressure_level,latitude,longitude,expver,divergence,cloud_cover,geopotential,ozone,potential_vorticity,relative_humidity,cloud_ice_water_content,cloud_liquid_water_content,rain_water_content,snow_water_content,temperature,u_component_wind,v_component_wind,vertical_velocity,relative_velocity
0,1577880000,1000.0,35.92,-17.00,1,0.000017,0.0,2481.8625,8.596089e-08,1.688022e-09,77.631280,0.0,0.0,0.000000e+00,0.0,287.75684,-2.724360,5.698472,0.006950,-0.000022
1,1577880000,1000.0,35.92,-16.75,1,0.000013,0.0,2495.2688,8.589063e-08,2.910383e-10,77.639090,0.0,0.0,0.000000e+00,0.0,287.60718,-2.625728,5.362046,-0.012001,-0.000021
2,1577880000,1000.0,35.92,-16.50,1,0.000008,0.0,2507.8938,8.583174e-08,2.852175e-09,78.129326,0.0,0.0,1.990702e-07,0.0,287.36280,-2.516353,4.990464,-0.015694,-0.000026
3,1577880000,1000.0,35.92,-16.25,1,0.000007,0.0,2519.2375,8.581856e-08,2.852175e-09,81.090260,0.0,0.0,3.886235e-07,0.0,287.02930,-2.448970,4.623277,-0.016609,-0.000020
4,1577880000,1000.0,35.92,-16.00,1,0.000009,0.0,2529.7063,8.582856e-08,-8.731149e-10,83.584404,0.0,0.0,5.149923e-07,0.0,286.69556,-2.378169,4.425767,-0.012978,-0.000014


In [5]:
train_data.describe()

,valid_time,pressure_level,latitude,longitude,expver,divergence,cloud_cover,geopotential,ozone,potential_vorticity,relative_humidity,cloud_ice_water_content,cloud_liquid_water_content,rain_water_content,snow_water_content,temperature,u_component_wind,v_component_wind,vertical_velocity,relative_velocity
count,1.461135e+06,1461135.0,1.461135e+06,1.461135e+06,1461135.0,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06,1.461135e+06
mean,1.640820e+09,1000.0,2.867000e+01,-9.000000e+00,1.0,1.722278e-06,1.087433e-03,1.419273e+03,7.026387e-08,-5.426510e-07,4.189150e+01,6.715325e-11,1.297110e-07,2.289065e-07,1.065386e-08,2.983226e+02,-1.515859e+00,-2.123802e+00,-4.008099e-02,2.670519e-07
std,3.649022e+07,0.0,4.257348e+00,4.690417e+00,0.0,2.898915e-05,2.060328e-02,4.259251e+02,1.060498e-08,1.057835e-06,2.827457e+01,9.289586e-09,3.248432e-06,3.524943e-06,1.078270e-06,8.534992e+00,4.077616e+00,4.280313e+00,1.295617e-01,3.893665e-05
min,1.577880e+09,1000.0,2.142000e+01,-1.700000e+01,1.0,-3.748042e-04,0.000000e+00,-1.565844e+02,1.669863e-08,-2.261421e-05,1.829086e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.801143e+02,-2.431671e+01,-2.212109e+01,-1.856550e+00,-4.818798e-04
25%,1.609243e+09,1000.0,2.492000e+01,-1.300000e+01,1.0,-9.511867e-06,0.000000e+00,1.117577e+03,6.514128e-08,-9.664882e-07,1.627430e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.916555e+02,-4.445672e+00,-4.922210e+00,-5.931404e-02,-1.781601e-05
50%,1.640779e+09,1000.0,2.867000e+01,-9.000000e+00,1.0,3.682566e-06,0.000000e+00,1.401596e+03,7.199844e-08,-4.888543e-07,3.229046e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.956709e+02,-1.647333e+00,-1.929118e+00,-1.901054e-02,-4.477130e-06
75%,1.672315e+09,1000.0,3.242000e+01,-5.000000e+00,1.0,1.537227e-05,0.000000e+00,1.700671e+03,7.740482e-08,-5.476977e-09,7.053055e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.045442e+02,1.182203e+00,6.653738e-01,1.298505e-02,1.325613e-05
max,1.703851e+09,1000.0,3.592000e+01,-1.000000e+00,1.0,3.991201e-04,1.000000e+00,2.984409e+03,9.684786e-08,3.533963e-05,1.182185e+02,3.892928e-06,5.402565e-04,7.351488e-04,2.839565e-04,3.202366e+02,2.045325e+01,1.910712e+01,3.066428e+00,5.883018e-04


In [6]:
train_data.isnull().sum()


valid_time                    0
pressure_level                0
latitude                      0
longitude                     0
expver                        0
divergence                    0
cloud_cover                   0
geopotential                  0
ozone                         0
potential_vorticity           0
relative_humidity             0
cloud_ice_water_content       0
cloud_liquid_water_content    0
rain_water_content            0
snow_water_content            0
temperature                   0
u_component_wind              0
v_component_wind              0
vertical_velocity             0
relative_velocity             0
dtype: int64